In [1]:
from kafka import KafkaAdminClient
from kafka.protocol.group import MemberAssignment
from kafka import KafkaConsumer, TopicPartition
import pandas as pd

In [2]:
group_id = 'reco-worker-oplog'
topic = 'oplog'

In [ ]:
result = []
bootstrap_servers=[f'cdh-work-0{i:02}:9092' for i in range(1, 11)]
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
consumer = KafkaConsumer(topic,
                         bootstrap_servers=bootstrap_servers,
                         group_id=group_id)
for member in admin_client.describe_consumer_groups([group_id])[0][5]:
    for data in MemberAssignment.decode(member[4]).assignment:
        topic = data[0]
        parts = data[1]
        for part in parts:
            topart = TopicPartition(topic, part)
            last_offsets = consumer.end_offsets([topart])
            committed = consumer.committed(topart)
            if last_offsets is not None and committed is not None:
                offset = last_offsets.get(topart)
                lag = offset - committed
                result.append({
                    'topic': topic,
                    'group': group_id,
                    'partition': part,
                    'committed': committed,
                    'current_offset': offset,
                    'lag': lag
                })

In [ ]:
df = pd.DataFrame(result)
display(df.head())
display(df[['lag']].describe())